# This notebook is the start of the "trial and error approach"

References:

- https://github.com/xurxodiz/cardwalker/tree/master/oracle
- https://laterna--magica.blogspot.com/2011/10/oracle-parser.html

In [ ]:
import json
import nltk
import pandas as pd
import re
from collections import defaultdict

In [ ]:
sets = json.load(open('./AllSets.json', 'rb'))

In [ ]:
for k, v in sorted(sets.items()):
    print(k, v['name'])

In [ ]:
cards_usaga = sets['USG']['cards']

In [ ]:
cards_usaga

# Let's start by trying to extract static habilities from cards

In [ ]:
cards_df = pd.DataFrame.from_dict(cards_usaga)

In [ ]:
texts = [card['text'].replace(card['name'], 'SELF') for card in cards_usaga if 'text' in card.keys()]

In [ ]:
patterns = [
    (r'^([A-Za-z]+ ?[A-Za-z]+)[$|\n|,]| \(', 'STATICABILITY'),
    (r', ([A-Za-z]+ ?[A-Za-z]+)[$|\n||,]| \(', 'STATICABILITY'),
]

In [ ]:
#regexp_tagger = nltk.RegexpTagger(patterns)

In [ ]:
res = defaultdict(list)
r=None
for text in texts:
#     if r: break
    for pat, tag in patterns:
        r = re.search(pat, text)
        if r:
            res[text].append((r.groups(), tag))
#             break
pretty = pd.DataFrame.from_dict(res, orient='index')
pretty

# DF version: Let's start by trying to extract static habilities from cards

In [ ]:
cards_df['text_preworked'] = cards_df.apply(lambda x: str(x['text']).replace(x['name'], 'SELF'), axis=1)

In [ ]:
cards_df.head(4).transpose()

In [ ]:
#regexp_tagger = nltk.RegexpTagger(patterns)

In [ ]:
patterns_static_abilities = [
    (r'^(?!(Enchant|Choose))([A-Z][a-z]+ ?[a-z]+)[$|\n|,]| \(', 'STATICABILITY'),
    (r', ([A-Z][a-z]+ ?[a-z]+)[$|\n||,]| \(', 'STATICABILITY'),
]

def detect_staticabilities(text):
    res = []# defaultdict(list)
    r = None
    for pat, tag in patterns_static_abilities:
        r = re.search(pat, text)
        if r:
            for group in r.groups():
                if group: res.append(group)
    
    return tuple(res) or pd.np.nan

cards_df['static_abilities'] = cards_df['text_preworked'].apply(detect_staticabilities)
cards_df.head(10).transpose()

In [ ]:
cards_df.dropna(subset=['static_abilities']).transpose()

In [ ]:
cards_df.iloc[288].text

In [ ]:
cards_df.dropna(subset=['static_abilities'])['static_abilities'].unique()